# **Network Data Source**: Label Prop
------------------

In [1]:
import pandas as pd, numpy as np, networkx as nx
import matplotlib.pyplot as plt
import requests
from openhunt import ossem, descriptiveStatistics as ds, visualizations as vis
import warnings
warnings.filterwarnings('ignore')

# Import packages
import os
import seaborn as sns
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
%matplotlib inline

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import chi2
from sklearn.linear_model.logistic import LogisticRegression
pd.options.display.max_columns = 500
pd.options.display.max_colwidth = 500

In [3]:
#!wget https://raw.githubusercontent.com/OTRF/detection-hackathon-apt29/master/datasets/day1/zeek/combined_zeek.log
#!wget https://raw.githubusercontent.com/hunters-forge/mordor/master/datasets/large/apt29/day1/zeek/combined_zeek.log

In [4]:
# Reference: https://infosecjupyterthon.com/notebooks/2_Jupyterthon_Cyb3rPandaH_2020.html
df_zeek = pd.read_json("combined_zeek.log.1", lines = True)

print(type(df_zeek))
df_zeek.head()

<class 'pandas.core.frame.DataFrame'>


,@stream,@system,@proc,ts,uid,id_orig_h,id_orig_p,id_resp_h,id_resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,orig_l2_addr,resp_l2_addr,rtt,named_pipe,endpoint,operation,trans_id,query,qclass,qclass_name,qtype,qtype_name,rcode,rcode_name,AA,TC,RD,RA,Z,answers,TTLs,rejected,analyzer,failure_reason,fuid,tx_hosts,rx_hosts,conn_uids,source,depth,analyzers,mime_type,filename,is_orig,seen_bytes,total_bytes,missing_bytes,overflow_bytes,timedout,md5,sha1,request_type,client,success,error_msg,till,forwardable,renewable,cipher,note,msg,sub,src,dst,p,actions,suppress_for,action,path,name,size,times_modified,times_accessed,times_created,times_changed,share_type,version,curve,server_name,resumed,established,cert_chain_fuids,client_cert_chain_fuids,subject,issuer,validation_status,resp_certificate_sha1,ja3,ja3s,notice,peer,addl,id,certificate_version,certificate_serial,certificate_subject,certificate_issuer,certificate_not_valid_before,certificate_not_valid_after,certificate_key_alg,certificate_sig_alg,certificate_key_type,certificate_key_length,certificate_exponent,basic_constraints_ca,extracted,extracted_cutoff,trans_depth,method,host,uri,user_agent,request_body_len,response_body_len,status_code,status_msg,tags,resp_fuids,resp_mime_types,client_header_names,orig_fuids,orig_mime_types,machine,compile_ts,os,subsystem,is_exe,is_64bit,uses_aslr,uses_dep,uses_code_integrity,uses_seh,has_import_table,has_export_table,has_cert_table,has_debug_data,section_names
0,conn,bobs.bigwheel.local,zeek,1.588205e+09,Cvf4XX17hSAgXDdGEd,10.0.1.6,54243.0,10.0.0.4,53.0,udp,dns,0.001528,39.0,55.0,SF,0.0,Dd,1.0,67.0,1.0,83.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,conn,bobs.bigwheel.local,zeek,1.588205e+09,CJ21Le4zsTUcyKKi98,10.0.1.6,56880.0,10.0.0.4,445.0,tcp,"gssapi,smb,krb",10.761077,4020.0,1681.0,RSTO,0.0,ShADdaR,12.0,1307.0,11.0,2133.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,conn,bobs.bigwheel.local,zeek,1.588205e+09,CnOP7t1eGGHf6LFfuk,10.0.1.6,65108.0,10.0.0.4,53.0,udp,dns,0.001599,36.0,104.0,SF,0.0,Dd,1.0,64.0,1.0,132.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,conn,bobs.bigwheel.local,zeek,1.588205e+09,CvxbPE3MuO7boUdSc8,10.0.1.6,138.0,10.0.1.255,138.0,udp,NaN,NaN,NaN,NaN,S0,0.0,D,1.0,229.0,0.0,0.0,00:0d:3a:13:e3:69,ff:ff:ff:ff:ff:ff,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [5]:
df_label = pd.read_json('./df_labelled_coalesce.json', lines=True)
#df_label = pd.read_json('./df_TP_Binary.json', lines=True)
print(type(df_label))
df_label.head()

<class 'pandas.core.frame.DataFrame'>


,@timestamp,@version,Category,Channel,EventID,EventReceivedTime,EventTime,EventType,ExecutionProcessID,Hostname,IpAddress,IpPort,Keywords,LogonGuid,Message,Opcode,OpcodeValue,ProviderGuid,RecordNumber,ServiceName,ServiceSid,Severity,SeverityValue,SourceModuleName,SourceModuleType,SourceName,Status,TargetDomainName,TargetUserName,Task,ThreadID,TicketEncryptionType,TicketOptions,TransmittedServices,Version,host,port,tags,TP_Label,Action,ActivityID,CalloutKey,CalloutName,ChangeType,Conditions,FilterId,FilterKey,FilterName,FilterType,LayerId,LayerKey,LayerName,ProcessId,ProviderKey,ProviderName,UserName,UserSid,Weight,AccessList,AccessMask,AccessReason,HandleId,ObjectName,ObjectServer,ObjectType,PrivilegeList,ProcessName,ResourceAttributes,RestrictedSidCount,SubjectDomainName,SubjectLogonId,SubjectUserName,SubjectUserSid,TransactionId,ShareLocalPath,ShareName,RelativeTargetName,CommandLine,MandatoryLabel,NewProcessId,NewProcessName,ParentProcessName,TargetLogonId,TargetUserSid,TokenElevationType,Service,DisabledPrivilegeList,EnabledPrivilegeList,CallerProcessId,CallerProcessName,TargetSid,AuthenticationPackageName,ElevatedToken,ImpersonationLevel,KeyLength,LmPackageName,LogonProcessName,LogonType,RestrictedAdminMode,TargetLinkedLogonId,TargetOutboundDomainName,TargetOutboundUserName,VirtualAccount,WorkstationName,SourceHandleId,SourceProcessId,TargetHandleId,TargetProcessId,AccountName,AccountType,ContextInfo,Domain,Payload,UserID,NewThreadId,RuleName,SourceImage,SourceProcessGuid,StartAddress,StartFunction,StartModule,TargetImage,TargetProcessGuid,UtcTime,MessageNumber,MessageTotal,Path,ScriptBlockId,ScriptBlockText,AlgorithmName,KeyName,KeyType,Operation,ReturnCode,Archived,Hashes,Image,IsExecutable,ProcessGuid,TargetFilename,User,CreationUtcTime,Company,Description,FileVersion,ImageLoaded,OriginalFileName,Product,Signature,SignatureStatus,Signed,DestinationHostname,DestinationIp,DestinationIsIpv6,DestinationPort,DestinationPortName,Initiated,Protocol,SourceHostname,SourceIp,SourceIsIpv6,SourcePort,SourcePortName,NewSd,OldSd,PipeName,CurrentDirectory,IntegrityLevel,LogonId,ParentCommandLine,ParentImage,ParentProcessGuid,ParentProcessId,TerminalSessionId,CallTrace,GrantedAccess,SourceProcessGUID,SourceThreadId,TargetProcessGUID,Device,TargetObject,Details,Application,FilterRTID,LayerRTID,SourceAddress,DestAddress,DestPort,Direction,RemoteMachineID,RemoteUserID,BootMenuPolicy,NewValue,NewValueType,ObjectValueName,OldValue,OldValueType,OperationType,ProfileUsed,RuleId,QueryName,QueryResults,QueryStatus,PreviousCreationUtcTime,EventCountTotal,EventIdx,GroupMembership,param1,Properties,Hash,PreAuthType,ClientProcessId,CountOfCredentialsReturned,ProcessCreationTime,ReadOperation,TargetName,Type,CalloutId,CalloutType,CorruptionActionState,DeviceName,DriveName,Config,IsTestConfig,DirtyPages,HiveName,HiveNameLength,KeysUpdated,PackageName,Workstation,DeviceNameLength,DeviceTime,DeviceVersionMajor,DeviceVersionMinor,FinalStatus,UserData,ServiceAccount,ServiceFileName,ServiceStartType,ServiceType,param10,param11,param2,param3,param4,param5,param6,param7,param8,param9,SubLayerKey,SubLayerName,SubLayerType,Default SD String:,ClassId,ClassName,CompatibleIds,DeviceDescription,DeviceId,LocationInformation,VendorIds,TargetProcessName,SecurityPackageName,AdditionalInfo,AdditionalInfo2,ProviderType,RuleAttr,AdapterName,AdapterSuffixName,DnsServerList,ErrorCode,Ipaddress,Sent UpdateServer,TaskContentNew,TaskName,ActiveProfile,MiniportName,MiniportNameLen,GroupPolicyApplied,LogDroppedPacketsEnabled,LogSuccessfulConnectionsEnabled,MulticastFlowsEnabled,OperationMode,Profile,RemoteAdminEnabled,TargetInfo,TargetLogonGuid,TargetServerName,ClientCreationTime,TimeSource,ImagePath,StartType,ProfileChanged,BootType,LoadOptions,EntryCount,KeyFilePath,ProviderContextKey,ProviderContextName,ProviderContextType,DwordVal,InterfaceGuid,InterfaceName,NewProfile,OldProfile,InstanceId,MaxRunspaces,MinRunspaces,CurrentStratumNumber,TimeSourceRefId,ReasonForRejection,

In [6]:
df_label['TP_Binary'] = df_label.TP_Label!='0'
df_label.replace('-', np.nan, inplace=True)

In [7]:
df_label.TP_Binary.unique()

array([False,  True])

In [8]:
# Host Data with Network Keys and Event IDs
# Network Keys and Event IDs
#Sysmon Event ID 3 3: Network connection
#4624(S): An account was successfully logged on.
#4648(S): A logon was attempted using explicit credentials.
#4768(S, F): A Kerberos authentication ticket (TGT) was requested.
#4769(S, F): A Kerberos service ticket was requested.
#5140(S, F): A network share object was accessed.
#5145(S, F): A network share object was checked to see whether client can be granted desired access.
#5154(S): The Windows Filtering Platform has permitted an application or service to listen on a port for incoming connections.
#5156(S): The Windows Filtering Platform has permitted a connection.
#5157(F): The Windows Filtering Platform has blocked a connection.
#5158(S): The Windows Filtering Platform has permitted a bind to a local port.

df_label[df_label.IpAddress.notnull() |
         df_label.DestinationIp.notnull() | 
         df_label.DestAddress.notnull() |
        df_label.SourceIp.notnull() | 
        df_label.SourceAddress.notnull()].groupby(['EventID'])['EventID', 'IpAddress', 'SourceIp', 'SourceAddress', 'IpPort', 'SourcePort', 'DestinationIp', 'DestAddress', 'DestinationPort', 'DestPort'].count()

,EventID,IpAddress,SourceIp,SourceAddress,IpPort,SourcePort,DestinationIp,DestAddress,DestinationPort,DestPort
EventID,,,,,,,,,,
3,1229,0,1229,0,0,1229,1229,0,1229,0
4624,180,180,0,0,180,0,0,0,0,0
4648,5,5,0,0,5,0,0,0,0,0
4768,14,14,0,0,14,0,0,0,0,0
4769,28,28,0,0,28,0,0,0,0,0
5140,42,42,0,0,42,0,0,0,0,0
5145,121,121,0,0,121,0,0,0,0,0
5154,362,0,0,362,0,362,0,0,0,0
5156,3163,0,0,3163,0,3163,0,3163,0,3163


In [9]:
# Transform Similar Data to Single Columns
df_label['Destination_IP'] = df_label['DestinationIp']
df_label['Destination_IP'] = df_label['Destination_IP'].fillna(df_label['DestAddress'])
df_label['Source_IP'] = df_label['IpAddress']
df_label['Source_IP'] = df_label['Source_IP'].fillna(df_label['SourceIp'])
df_label['Source_IP'] = df_label['Source_IP'].fillna(df_label['SourceAddress'])
df_label['Source_Port'] = df_label['IpPort']
df_label['Source_Port'] = df_label['Source_Port'].fillna(df_label['SourcePort'])
df_label['Destination_Port'] = df_label['DestinationPort']
df_label['Destination_Port'] = df_label['Destination_Port'].fillna(df_label['DestPort'])

In [10]:
df_label[df_label.Source_IP.notnull() |
         df_label.Destination_IP.notnull() | 
        df_label.Source_Port.notnull() | 
        df_label.Destination_Port.notnull()].groupby(['EventID'])['EventID', 'Source_IP', 'Source_Port', 'Destination_IP', 'Destination_Port'].count()

,EventID,Source_IP,Source_Port,Destination_IP,Destination_Port
EventID,,,,,
3,1229,1229,1229,1229,1229
4624,180,180,180,0,0
4648,5,5,5,0,0
4768,14,14,14,0,0
4769,28,28,28,0,0
5140,42,42,42,0,0
5145,121,121,121,0,0
5154,362,362,362,0,0
5156,3163,3163,3163,3163,3163


In [11]:
df_zeek.shape

(2140, 148)

In [12]:
df_zeek[df_zeek.id_orig_h.notnull() |
         df_zeek.id_orig_p.notnull() | 
        df_zeek.id_resp_h.notnull() | 
        df_zeek.id_resp_p.notnull()].groupby(['id_orig_h', 'id_orig_p', 'id_resp_h', 'id_orig_p'])['@stream', 'id_orig_h', 'id_orig_p', 'id_resp_h', 'id_resp_p'].count()

@stream  id_orig_h  id_orig_p  \
id_orig_h id_orig_p id_resp_h  id_orig_p                                  
10.0.1.4  123.0     10.0.0.4   123.0            3          3          3   
          137.0     10.0.1.255 137.0            4          4          4   
          138.0     10.0.1.255 138.0            4          4          4   
          50051.0   10.0.0.4   50051.0          3          3          3   
          51183.0   10.0.0.4   51183.0          2          2          2   
...                                           ...        ...        ...   
10.0.1.6  64173.0   10.0.0.4   64173.0          2          2          2   
          64562.0   10.0.0.4   64562.0          2          2          2   
          64563.0   10.0.0.4   64563.0          1          1          1   
          64909.0   10.0.0.4   64909.0          2          2          2   
          65108.0   10.0.0.4   65108.0          2          2          2   

                                          id_resp_h  id_resp_p  
id_orig_h id_orig_p id_resp_h  id_orig_p                        
10.0.1.4  123.0     10.0.0.4   123.0              3          3  
          137.0     10.0.1.255 137.0              4          4  
          138.0     10.0.1.255 138.0              4          4  
          50051.0   10.0.0.4   50051.0            3          3  
          51183.0   10.0.0.4   51183.0            2          2  
...                                             ...        ...  
10.0.1.6  64173.0   10.0.0.4   64173.0            2          2  
          64562.0   10.0.0.4   64562.0            2          2  
          64563.0   10.0.0.4   64563.0            1          1  
          64909.0   10.0.0.4   64909.0            2          2  
          65108.0   10.0.0.4   65108.0            2          2  

[537 rows x 5 columns]

In [13]:
df_zeek[df_zeek.id_orig_h.notnull() |
         df_zeek.id_orig_p.notnull() | 
        df_zeek.id_resp_h.notnull() | 
        df_zeek.id_resp_p.notnull()].groupby(['@stream'])['@stream', 'id_orig_h', 'id_orig_p', 'id_resp_h', 'id_resp_p'].count()

,@stream,id_orig_h,id_orig_p,id_resp_h,id_resp_p
@stream,,,,,
conn,613,613,613,613,613
dce_rpc,123,123,123,123,123
dns,53,53,53,53,53
dpd,12,12,12,12,12
http,62,62,62,62,62
kerberos,35,35,35,35,35
notice,2,2,2,2,2
smb_files,12,12,12,12,12
smb_mapping,11,11,11,11,11


In [14]:
df_zeek['Source_IP'] = df_zeek['id_orig_h']
df_zeek['Source_Port'] = df_zeek['id_orig_p']
df_zeek['Destination_IP'] = df_zeek['id_resp_h']
df_zeek['Destination_Port'] = df_zeek['id_resp_p']

In [15]:
df_zeek[df_zeek.id_orig_h.notnull() |
         df_zeek.id_orig_p.notnull() | 
        df_zeek.id_resp_h.notnull() | 
        df_zeek.id_resp_p.notnull()].groupby(['@stream'])['@stream', 'Source_IP', 'Source_Port', 'Destination_IP', 'Destination_Port'].count()

,@stream,Source_IP,Source_Port,Destination_IP,Destination_Port
@stream,,,,,
conn,613,613,613,613,613
dce_rpc,123,123,123,123,123
dns,53,53,53,53,53
dpd,12,12,12,12,12
http,62,62,62,62,62
kerberos,35,35,35,35,35
notice,2,2,2,2,2
smb_files,12,12,12,12,12
smb_mapping,11,11,11,11,11


In [16]:
df_label = df_label.astype({'Destination_Port': 'object'})
df_label.dtypes

@timestamp          object
@version             int64
Category            object
Channel             object
EventID              int64
                     ...  
TP_Binary             bool
Destination_IP      object
Source_IP           object
Source_Port         object
Destination_Port    object
Length: 394, dtype: object

In [17]:
df_zeek = df_zeek.astype({'Source_Port': 'object'})
df_zeek = df_zeek.astype({'Destination_Port': 'object'})
df_zeek.dtypes

@stream              object
@system              object
@proc                object
ts                  float64
uid                  object
                     ...   
section_names        object
Source_IP            object
Source_Port          object
Destination_IP       object
Destination_Port     object
Length: 152, dtype: object

In [18]:
# Test the join keys

#df_zeek[df_zeek.id_orig_h.notnull() |
#         df_zeek.id_orig_p.notnull() | 
#        df_zeek.id_resp_h.notnull() | 
#        df_zeek.id_resp_p.notnull()].groupby(['@system', '@stream'])['@stream', 'Source_IP', 'Source_Port', 'Destination_IP', 'Destination_Port'].count()

df_zeek[df_zeek.id_orig_h.notnull() |
         df_zeek.id_orig_p.notnull() | 
        df_zeek.id_resp_h.notnull() | 
        df_zeek.id_resp_p.notnull()].groupby(['ts', '@system', 'Source_IP', 'Source_Port', 'Destination_IP', 'Destination_Port'])['ts', 'Source_IP', 'Source_Port', 'Destination_IP', 'Destination_Port'].count()

ts  \
ts           @system                        Source_IP Source_Port Destination_IP Destination_Port       
1.588205e+09 simplerockbuild.simplerock.lan 10.0.1.4  64842.0     10.0.0.4       53.0               2   
                                                      60595.0     10.0.0.4       445.0              1   
                                                                                 445.0              1   
                                                                                 445.0              1   
             bobs.bigwheel.local            10.0.1.6  54243.0     10.0.0.4       53.0               2   
...                                                                                                ..   
1.588207e+09 bobs.bigwheel.local            10.0.1.4  60879.0     10.0.1.6       55121.0            1   
             simplerockbuild.simplerock.lan 10.0.1.4  60879.0     10.0.1.6       55121.0            1   
1.588208e+09 bobs.bigwheel.local            10.0.1.4  60879.0     10.0.1.6       55121.0            1   
             simplerockbuild.simplerock.lan 10.0.1.4  60879.0     10.0.1.6       55121.0            1   
                                                      138.0       10.0.1.255     138.0              1   

                                                                                                   Source_IP  \
ts           @system                        Source_IP Source_Port Destination_IP Destination_Port              
1.588205e+09 simplerockbuild.simplerock.lan 10.0.1.4  64842.0     10.0.0.4       53.0                      2   
                                                      60595.0     10.0.0.4       445.0                     1   
                                                                                 445.0                     1   
                                                                                 445.0                     1   
             bobs.bigwheel.local            10.0.1.6  54243.0     10.0.0.4       53.0                      2   
...                                                                                                      ...   
1.588207e+09 bobs.bigwheel.local            10.0.1.4  60879.0     10.0.1.6       55121.0                   1   
             simplerockbuild.simplerock.lan 10.0.1.4  60879.0     10.0.1.6       55121.0                   1   
1.588208e+09 bobs.bigwheel.local            10.0.1.4  60879.0     10.0.1.6       55121.0                   1   
             simplerockbuild.simplerock.lan 10.0.1.4  60879.0     10.0.1.6       55121.0                   1   
                                                      138.0       10.0.1.255     138.0                     1   

                                                                                                   Source_Port  \
ts           @system                        Source_IP Source_Port Destination_IP Destination_Port                
1.588205e+09 simplerockbuild.simplerock.lan 10.0.1.4  64842.0     10.0.0.4       53.0                        2   
                                                      60595.0     10.0.0.4       445.0                       1   
                                                                                 445.0                       1   
                                                                                 445.0                       1   
             bobs.bigwheel.local            10.0.1.6  54243.0     10.0.0.4       53.0                        2   
...                                                                                                        ...   
1.588207e+09 bobs.bigwheel.local            10.0.1.4  60879.0     10.0.1.6       55121.0                     1   
             simplerockbuild.simplerock.lan 10.0.1.4  60879.0     10.0.1.6       55121.0                     1   
1.588208e+09 bobs.bigwheel.local            10.0.1.4  60879.0     10.0.1.6       55121.0                     1   
             simpleroc

In [19]:
df_label[df_label.Source_IP.notnull() |
         df_label.Source_Port.notnull() | 
        df_label.Destination_IP.notnull() | 
        df_label.Destination_Port.notnull()].groupby(['@timestamp', 'Hostname', 'Source_IP', 'Source_Port', 'Destination_IP', 'Destination_Port'])['@timestamp', 'Source_IP', 'Source_Port', 'Destination_IP', 'Destination_Port'].count()

@timestamp  \
@timestamp               Hostname               Source_IP   Source_Port Destination_IP  Destination_Port               
2020-05-02T02:55:27.499Z NEWYORK.dmevals.local  10.0.0.4    49641.0     64.4.48.201     53.0                       1   
                                                            50505.0     40.90.4.4       53.0                       1   
                                                172.18.39.2 63466.0     10.0.0.4        53.0                       1   
2020-05-02T02:55:28.511Z NEWYORK.dmevals.local  10.0.0.4    53.0        172.18.39.2     63466.0                    1   
                         SCRANTON.dmevals.local 10.0.1.4    49702.0     169.254.169.254 80.0                       1   
...                                                                                                              ...   
2020-05-02T03:28:00.015Z NEWYORK.dmevals.local  10.0.0.4    57354.0     169.254.169.254 80.0                       1   
2020-05-02T03:28:01.018Z UTICA.dmevals.local    10.0.1.5    49706.0     169.254.169.254 80.0                       1   
2020-05-02T03:28:01.019Z UTICA.dmevals.local    10.0.1.5    49706.0     169.254.169.254 80.0                       1   
2020-05-02T03:28:11.111Z UTICA.dmevals.local    10.0.1.5    49706.0     169.254.169.254 80.0                       1   
2020-05-02T03:28:11.112Z UTICA.dmevals.local    10.0.1.5    49706.0     169.254.169.254 80.0                       1   

                                                                                                          Source_IP  \
@timestamp               Hostname               Source_IP   Source_Port Destination_IP  Destination_Port              
2020-05-02T02:55:27.499Z NEWYORK.dmevals.local  10.0.0.4    49641.0     64.4.48.201     53.0                      1   
                                                            50505.0     40.90.4.4       53.0                      1   
                                                172.18.39.2 63466.0     10.0.0.4        53.0                      1   
2020-05-02T02:55:28.511Z NEWYORK.dmevals.local  10.0.0.4    53.0        172.18.39.2     63466.0                   1   
                         SCRANTON.dmevals.local 10.0.1.4    49702.0     169.254.169.254 80.0                      1   
...                                                                                                             ...   
2020-05-02T03:28:00.015Z NEWYORK.dmevals.local  10.0.0.4    57354.0     169.254.169.254 80.0                      1   
2020-05-02T03:28:01.018Z UTICA.dmevals.local    10.0.1.5    49706.0     169.254.169.254 80.0                      1   
2020-05-02T03:28:01.019Z UTICA.dmevals.local    10.0.1.5    49706.0     169.254.169.254 80.0                      1   
2020-05-02T03:28:11.111Z UTICA.dmevals.local    10.0.1.5    49706.0     169.254.169.254 80.0                      1   
2020-05-02T03:28:11.112Z UTICA.dmevals.local    10.0.1.5    49706.0     169.254.169.254 80.0                      1   

                                                                                                          Source_Port  \
@timestamp               Hostname               Source_IP   Source_Port Destination_IP  Destination_Port                
2020-05-02T02:55:27.499Z NEWYORK.dmevals.local  10.0.0.4    49641.0     64.4.48.201     53.0                        1   
                                                            50505.0     40.90.4.4       53.0                        1   
                                                172.18.39.2 63466.0     10.0.0.4        53.0                        1   
2020-05-02T02:55:28.511Z NEWYORK.dmevals.local  10.0.0.4    53.0        172.18.39.2     63466.0                     1   
                         SCRANTON.dmevals.local 10.0.1.4    49702.0     169.254.169.254 80.0                        1   
...                                                                                                               ...   
2020-05-02T03:28:00.015Z NEWY

In [20]:
# MergeError: Merge keys are not unique in either left or right dataset; not a one-to-one merge
# Need to add a unique key in both datasets -> Timestamp? @system=bobs.bigwheel.local/Hostname
#result = pd.merge(df_label, df_zeek,
#                  on=['Source_IP', 'Source_Port', 'Destination_IP', 'Destination_Port'],
#                  how='outer',
#                  validate="one_to_one")

In [21]:
# Kernel crash
# Need to add a unique key in both datasets -> Timestamp? @system=bobs.bigwheel.local/Hostname
#result = pd.merge(df_label, df_zeek,
#                  on=['Source_IP', 'Source_Port', 'Destination_IP', 'Destination_Port'],
#                  how='outer',
#                  )

In [22]:
# Kernel crash
#result = pd.merge(df_label, df_zeek,
#                  on=['Source_IP', 'Source_Port', 'Destination_IP', 'Destination_Port'],
#                  how='left',
#                  )

In [23]:
# MemoryError: Unable to allocate 330. GiB for an array with shape (276, 160246003) and data type object
#df_zeek = pd.merge(df_label, df_zeek,
#                  on=['Source_IP', 'Source_Port', 'Destination_IP', 'Destination_Port'],
#                  how='right',
#                  )

In [24]:
df_zeek['ts_pd'] = df_zeek["ts"].apply(lambda x: (pd.to_datetime(x, unit='s', utc=True)))

In [25]:
df_label['ts_pd'] = df_label['@timestamp'].apply(lambda x: (pd.Timestamp(x, unit='s')))

In [26]:
# Find timestamp offset 
#    @timestamp 	Source_IP 	Source_Port 	Destination_IP 	Destination_Port
#    2020-05-02T02:55:27.499Z 	10.0.0.4 	49641.0 	64.4.48.201 	53.0

In [26]:
#df_zeek.loc[(df_zeek['@stream'] =='conn') & 
#            (df_zeek.Source_IP=="10.0.1.4") &
#            #(df_zeek.Source_Port=='123') #&
#            (df_zeek.Destination_IP=="10.0.0.4") #&
#            #(df_zeek.Destination_Port=="123")
#           ]

In [27]:
# bad_flows = df_label.loc[(df_label.TP_Binary==True)][['TP_Binary', '@timestamp', 'EventID', 'Source_IP', 'Source_Port', 'Destination_IP', 'Destination_Port']]
bad_flows = df_label[(df_label.TP_Binary==True) & (df_label.Source_IP.notnull())][['EventID', 'TP_Binary', 'TP_Label', '@timestamp', 'Source_IP', 'Source_Port', 'Destination_IP', 'Destination_Port']].drop_duplicates()

In [28]:
bad_flows

,EventID,TP_Binary,TP_Label,@timestamp,Source_IP,Source_Port,Destination_IP,Destination_Port
12637,5156,True,"0, 8.A.2. Remote System Discovery",2020-05-02T03:09:29.935Z,10.0.1.4,59961,10.0.1.6,5985
14743,3,True,"0, 8.A.1. Remote System Discovery",2020-05-02T03:09:07.649Z,10.0.1.4,59955,10.0.0.4,389
15466,5145,True,"0, 8.C.2. Windows Admin Shares",2020-05-02T03:12:47.917Z,10.0.1.4,59987,NaN,NaN
16280,5156,True,"0, 3.B.3. Commonly Used Port",2020-05-02T02:58:47.195Z,10.0.1.4,59845,192.168.0.5,443
16349,4624,True,"0, 8.C.1. Valid Accounts",2020-05-02T03:10:24.472Z,10.0.1.4,59967,NaN,NaN
24420,5145,True,"0, 8.B.1. Remote File Copy",2020-05-02T03:10:24.475Z,10.0.1.4,59967,NaN,NaN
24421,5145,True,"0, 8.C.2. Windows Admin Shares",2020-05-02T03:13:51.446Z,10.0.1.4,59997,NaN,NaN
24641,3,True,"0, 8.A.2. Remote System Discovery",2020-05-02T03:09:31.065Z,10.0.1.4,59960,10.0.1.6,5985
29064,5156,True,"0, 8.A.1. Remote System Discovery",2020-05-02T03:09:05.633Z,10.0.1.4,59955,10.0.0.4,389
29065,5156,True,"0, 8.A.2. Remote System Discovery",2020-05-02T03:09:32.125Z,10.0.1.4,59962,10.0.1.6,5985


In [29]:
#df_zeek2 = pd.merge(
#    left = df_zeek,
#    right = bad_flows,
#    how = 'left',
#    on = ['Source_IP', 'Source_Port', 'Destination_IP', 'Destination_Port']
#)

In [30]:
bad_flows.TP_Binary.value_counts()

True    36
Name: TP_Binary, dtype: int64

In [31]:
df_zeek = df_zeek.astype({'Source_Port': 'str'})
df_zeek = df_zeek.astype({'Source_IP': 'str'})
df_zeek = df_zeek.astype({'Destination_Port': 'str'})
df_zeek = df_zeek.astype({'Destination_IP': 'str'})

In [32]:
bad_flows = bad_flows.astype({'Source_Port': 'str'})
bad_flows = bad_flows.astype({'Source_IP': 'str'})
bad_flows = bad_flows.astype({'Destination_Port': 'str'})
bad_flows = bad_flows.astype({'Destination_IP': 'str'})

In [33]:
#df_zeek2 = pd.merge(
#    left = df_zeek,
#    right = bad_flows,
#    how = 'left',
#    on = ['Source_IP', 'Source_Port', 'Destination_IP', 'Destination_Port']
#)

In [34]:
# Transform Similar Data to Single Columns
#bad_flows['1_IP'] = bad_flows['Destination_IP']
#bad_flows['1_Port'] = bad_flows['Destination_Port']

#df_label['Destination_IP'] = df_label['Destination_IP'].fillna(df_label['DestAddress'])
#df_label['Source_IP'] = df_label['IpAddress']
#df_label['Source_IP'] = df_label['Source_IP'].fillna(df_label['SourceIp'])
#df_label['Source_IP'] = df_label['Source_IP'].fillna(df_label['SourceAddress'])
#df_label['Source_Port'] = df_label['IpPort']
#df_label['Source_Port'] = df_label['Source_Port'].fillna(df_label['SourcePort'])
#df_label['Destination_Port'] = df_label['Destination_Port'].fillna(df_label['DestPort'])

In [35]:
bad_flows.head()

,EventID,TP_Binary,TP_Label,@timestamp,Source_IP,Source_Port,Destination_IP,Destination_Port
12637,5156,True,"0, 8.A.2. Remote System Discovery",2020-05-02T03:09:29.935Z,10.0.1.4,59961.0,10.0.1.6,5985.0
14743,3,True,"0, 8.A.1. Remote System Discovery",2020-05-02T03:09:07.649Z,10.0.1.4,59955.0,10.0.0.4,389.0
15466,5145,True,"0, 8.C.2. Windows Admin Shares",2020-05-02T03:12:47.917Z,10.0.1.4,59987,nan,nan
16280,5156,True,"0, 3.B.3. Commonly Used Port",2020-05-02T02:58:47.195Z,10.0.1.4,59845.0,192.168.0.5,443.0
16349,4624,True,"0, 8.C.1. Valid Accounts",2020-05-02T03:10:24.472Z,10.0.1.4,59967,nan,nan


In [36]:
df_zeek.head()

,@stream,@system,@proc,ts,uid,id_orig_h,id_orig_p,id_resp_h,id_resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,orig_l2_addr,resp_l2_addr,rtt,named_pipe,endpoint,operation,trans_id,query,qclass,qclass_name,qtype,qtype_name,rcode,rcode_name,AA,TC,RD,RA,Z,answers,TTLs,rejected,analyzer,failure_reason,fuid,tx_hosts,rx_hosts,conn_uids,source,depth,analyzers,mime_type,filename,is_orig,seen_bytes,total_bytes,missing_bytes,overflow_bytes,timedout,md5,sha1,request_type,client,success,error_msg,till,forwardable,renewable,cipher,note,msg,sub,src,dst,p,actions,suppress_for,action,path,name,size,times_modified,times_accessed,times_created,times_changed,share_type,version,curve,server_name,resumed,established,cert_chain_fuids,client_cert_chain_fuids,subject,issuer,validation_status,resp_certificate_sha1,ja3,ja3s,notice,peer,addl,id,certificate_version,certificate_serial,certificate_subject,certificate_issuer,certificate_not_valid_before,certificate_not_valid_after,certificate_key_alg,certificate_sig_alg,certificate_key_type,certificate_key_length,certificate_exponent,basic_constraints_ca,extracted,extracted_cutoff,trans_depth,method,host,uri,user_agent,request_body_len,response_body_len,status_code,status_msg,tags,resp_fuids,resp_mime_types,client_header_names,orig_fuids,orig_mime_types,machine,compile_ts,os,subsystem,is_exe,is_64bit,uses_aslr,uses_dep,uses_code_integrity,uses_seh,has_import_table,has_export_table,has_cert_table,has_debug_data,section_names,Source_IP,Source_Port,Destination_IP,Destination_Port,ts_pd
0,conn,bobs.bigwheel.local,zeek,1.588205e+09,Cvf4XX17hSAgXDdGEd,10.0.1.6,54243.0,10.0.0.4,53.0,udp,dns,0.001528,39.0,55.0,SF,0.0,Dd,1.0,67.0,1.0,83.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0.1.6,54243.0,10.0.0.4,53.0,2020-04-30 00:06:39.824369907+00:00
1,conn,bobs.bigwheel.local,zeek,1.588205e+09,CJ21Le4zsTUcyKKi98,10.0.1.6,56880.0,10.0.0.4,445.0,tcp,"gssapi,smb,krb",10.761077,4020.0,1681.0,RSTO,0.0,ShADdaR,12.0,1307.0,11.0,2133.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0.1.6,56880.0,10.0.0.4,445.0,2020-04-30 00:06:39.827696085+00:00
2,conn,bobs.bigwheel.local,zeek,1.588205e+09,CnOP7t1eGGHf6LFfuk,10.0.1.6,65108.0,10.0.0.4,53.0,udp,dns,0.001599,36.0,104.0,SF,0.0,Dd,1.0,64.0,1.0,132.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0.1.6,65108.0,10.0.0.4,53.0,2020-04-30 00:07:59.953876019+00:00
3,conn,bobs.bigwheel.local,zeek,1.588205e+09,CvxbPE3MuO7boUdSc8,10.0.1.6,138.0,10.0.1.255,138.

In [37]:
bad_flows.dtypes

EventID              int64
TP_Binary             bool
TP_Label            object
@timestamp          object
Source_IP           object
Source_Port         object
Destination_IP      object
Destination_Port    object
dtype: object

In [38]:
df_zeek.dtypes

@stream                          object
@system                          object
@proc                            object
ts                              float64
uid                              object
                           ...         
Source_IP                        object
Source_Port                      object
Destination_IP                   object
Destination_Port                 object
ts_pd               datetime64[ns, UTC]
Length: 153, dtype: object

## Works!

In [39]:
#result = pd.merge(df_label, df_zeek,
#                  on=['Source_IP', 'Source_Port', 'Destination_IP', 'Destination_Port'],
#                  how='left',
#                  )

df_zeek2 = pd.merge(
    left = df_zeek,
    right = bad_flows,
    how = 'left',
    on = ['Destination_IP', 'Source_IP', 'Destination_Port']
)

In [40]:
df_zeek2.TP_Binary.value_counts()

True    134
Name: TP_Binary, dtype: int64

In [41]:
df_zeek2.shape

(2248, 158)

In [42]:
df_zeek.shape

(2140, 153)

In [43]:
df_zeek2.head()

,@stream,@system,@proc,ts,uid,id_orig_h,id_orig_p,id_resp_h,id_resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,orig_l2_addr,resp_l2_addr,rtt,named_pipe,endpoint,operation,trans_id,query,qclass,qclass_name,qtype,qtype_name,rcode,rcode_name,AA,TC,RD,RA,Z,answers,TTLs,rejected,analyzer,failure_reason,fuid,tx_hosts,rx_hosts,conn_uids,source,depth,analyzers,mime_type,filename,is_orig,seen_bytes,total_bytes,missing_bytes,overflow_bytes,timedout,md5,sha1,request_type,client,success,error_msg,till,forwardable,renewable,cipher,note,msg,sub,src,dst,p,actions,suppress_for,action,path,name,size,times_modified,times_accessed,times_created,times_changed,share_type,version,curve,server_name,resumed,established,cert_chain_fuids,client_cert_chain_fuids,subject,issuer,validation_status,resp_certificate_sha1,ja3,ja3s,notice,peer,addl,id,certificate_version,certificate_serial,certificate_subject,certificate_issuer,certificate_not_valid_before,certificate_not_valid_after,certificate_key_alg,certificate_sig_alg,certificate_key_type,certificate_key_length,certificate_exponent,basic_constraints_ca,extracted,extracted_cutoff,trans_depth,method,host,uri,user_agent,request_body_len,response_body_len,status_code,status_msg,tags,resp_fuids,resp_mime_types,client_header_names,orig_fuids,orig_mime_types,machine,compile_ts,os,subsystem,is_exe,is_64bit,uses_aslr,uses_dep,uses_code_integrity,uses_seh,has_import_table,has_export_table,has_cert_table,has_debug_data,section_names,Source_IP,Source_Port_x,Destination_IP,Destination_Port,ts_pd,EventID,TP_Binary,TP_Label,@timestamp,Source_Port_y
0,conn,bobs.bigwheel.local,zeek,1.588205e+09,Cvf4XX17hSAgXDdGEd,10.0.1.6,54243.0,10.0.0.4,53.0,udp,dns,0.001528,39.0,55.0,SF,0.0,Dd,1.0,67.0,1.0,83.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0.1.6,54243.0,10.0.0.4,53.0,2020-04-30 00:06:39.824369907+00:00,NaN,NaN,NaN,NaN,NaN
1,conn,bobs.bigwheel.local,zeek,1.588205e+09,CJ21Le4zsTUcyKKi98,10.0.1.6,56880.0,10.0.0.4,445.0,tcp,"gssapi,smb,krb",10.761077,4020.0,1681.0,RSTO,0.0,ShADdaR,12.0,1307.0,11.0,2133.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0.1.6,56880.0,10.0.0.4,445.0,2020-04-30 00:06:39.827696085+00:00,NaN,NaN,NaN,NaN,NaN
2,conn,bobs.bigwheel.local,zeek,1.588205e+09,CnOP7t1eGGHf6LFfuk,10.0.1.6,65108.0,10.0.0.4,53.0,udp,dns,0.001599,36.0,104.0,SF,0.0,Dd,1.0,64.0,1.0,132.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0.1.6,65108.0,10.0.0.4,53.0,2020-04-30 00:07:59.953876019+00:00,

In [44]:
df_zeek2[df_zeek2.TP_Binary == True]

,@stream,@system,@proc,ts,uid,id_orig_h,id_orig_p,id_resp_h,id_resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,orig_l2_addr,resp_l2_addr,rtt,named_pipe,endpoint,operation,trans_id,query,qclass,qclass_name,qtype,qtype_name,rcode,rcode_name,AA,TC,RD,RA,Z,answers,TTLs,rejected,analyzer,failure_reason,fuid,tx_hosts,rx_hosts,conn_uids,source,depth,analyzers,mime_type,filename,is_orig,seen_bytes,total_bytes,missing_bytes,overflow_bytes,timedout,md5,sha1,request_type,client,success,error_msg,till,forwardable,renewable,cipher,note,msg,sub,src,dst,p,actions,suppress_for,action,path,name,size,times_modified,times_accessed,times_created,times_changed,share_type,version,curve,server_name,resumed,established,cert_chain_fuids,client_cert_chain_fuids,subject,issuer,validation_status,resp_certificate_sha1,ja3,ja3s,notice,peer,addl,id,certificate_version,certificate_serial,certificate_subject,certificate_issuer,certificate_not_valid_before,certificate_not_valid_after,certificate_key_alg,certificate_sig_alg,certificate_key_type,certificate_key_length,certificate_exponent,basic_constraints_ca,extracted,extracted_cutoff,trans_depth,method,host,uri,user_agent,request_body_len,response_body_len,status_code,status_msg,tags,resp_fuids,resp_mime_types,client_header_names,orig_fuids,orig_mime_types,machine,compile_ts,os,subsystem,is_exe,is_64bit,uses_aslr,uses_dep,uses_code_integrity,uses_seh,has_import_table,has_export_table,has_cert_table,has_debug_data,section_names,Source_IP,Source_Port_x,Destination_IP,Destination_Port,ts_pd,EventID,TP_Binary,TP_Label,@timestamp,Source_Port_y
18,conn,bobs.bigwheel.local,zeek,1.588207e+09,C0XYv11LLkSOaKmcOa,10.0.1.4,60775.0,10.0.1.6,5985.0,tcp,NaN,0.011584,4719.0,426.0,SF,0.0,ShADadFf,9.0,5091.0,6.0,678.0,fc:bd:67:e1:a9:c1,00:0d:3a:13:e3:69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0.1.4,60775.0,10.0.1.6,5985.0,2020-04-30 00:33:29.274476051+00:00,5156.0,True,"0, 8.A.2. Remote System Discovery",2020-05-02T03:09:29.935Z,59961.0
19,conn,bobs.bigwheel.local,zeek,1.588207e+09,C0XYv11LLkSOaKmcOa,10.0.1.4,60775.0,10.0.1.6,5985.0,tcp,NaN,0.011584,4719.0,426.0,SF,0.0,ShADadFf,9.0,5091.0,6.0,678.0,fc:bd:67:e1:a9:c1,00:0d:3a:13:e3:69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0.1.4,60775.0,10.0.1.6,5985.0,2020-04-30 00:33:29.274476051+00:00,3.0,True,"0, 8.A.2. Remote System Discovery",2020-05-02T03:09:31.065Z,59960.0
20,conn,bobs.bigwheel.local,zeek,1.588207e+09,C0XYv11LLkSOaKmcOa,10.0.1.4,60775.0,10.0.1.6,5985.0,tcp,NaN,0.011584,4719.0,426.0,SF,0.0,ShADadFf,9.0,5091.0,6.0,678.0,fc:bd:67:e1:a9:c1,00:0d:3a:13:e3:69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [45]:
df_zeek2.Source_Port_y.value_counts()

59955.0    24
59957.0    24
59962.0    16
59959.0    16
59961.0    16
59960.0    16
59846.0    10
59845.0    10
59835.0     2
Name: Source_Port_y, dtype: int64

In [46]:
type(df_zeek2.Source_Port_x.value_counts())

pandas.core.series.Series

In [47]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df_zeek2.loc[df_zeek2.TP_Binary == True].Source_Port_x.value_counts())

60771.0    16
60775.0    16
60774.0    16
60772.0    16
60645.0    12
60646.0     8
60767.0     4
58800.0     4
60785.0     4
58560.0     4
58562.0     4
55871.0     4
60622.0     4
60765.0     4
60786.0     4
60787.0     4
60623.0     4
62150.0     4
60597.0     2
Name: Source_Port_x, dtype: int64


In [48]:
df_zeek2.TP_Binary = df_zeek2.TP_Binary.fillna(False)

In [49]:
df_zeek2.TP_Label = df_zeek2.TP_Label.fillna(0)

In [50]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_zeek2[df_zeek2.TP_Binary == True].groupby(['@system', '@timestamp', 'Source_IP', 'Source_Port_x', 'Destination_IP', 'Destination_Port'])['@timestamp', 'Source_IP', 'Source_Port_x', 'Destination_IP', 'Destination_Port'].count())

@timestamp  \
@system                        @timestamp               Source_IP Source_Port_x Destination_IP Destination_Port               
bobs.bigwheel.local            2020-05-02T03:09:24.813Z 10.0.1.4  60771.0       10.0.1.6       5985.0                     1   
                                                                  60772.0       10.0.1.6       5985.0                     1   
                                                                  60774.0       10.0.1.6       5985.0                     1   
                                                                  60775.0       10.0.1.6       5985.0                     1   
                               2020-05-02T03:09:27.836Z 10.0.1.4  60771.0       10.0.1.6       5985.0                     1   
                                                                  60772.0       10.0.1.6       5985.0                     1   
                                                                  60774.0       10.0.1.6       5985.0                     1   
                                                                  60775.0       10.0.1.6       5985.0                     1   
                               2020-05-02T03:09:28.850Z 10.0.1.4  60771.0       10.0.1.6       5985.0                     1   
                                                                  60772.0       10.0.1.6       5985.0                     1   
                                                                  60774.0       10.0.1.6       5985.0                     1   
                                                                  60775.0       10.0.1.6       5985.0                     1   
                               2020-05-02T03:09:29.935Z 10.0.1.4  60771.0       10.0.1.6       5985.0                     1   
                                                                  60772.0       10.0.1.6       5985.0                     1   
                                                                  60774.0       10.0.1.6       5985.0                     1   
                                                                  60775.0       10.0.1.6       5985.0                     1   
                               2020-05-02T03:09:31.065Z 10.0.1.4  60771.0       10.0.1.6       5985.0                     2   
                                                                  60772.0       10.0.1.6       5985.0                     2   
                                                                  60774.0       10.0.1.6       5985.0                     2   
                                                                  60775.0       10.0.1.6       5985.0                     2   
                               2020-05-02T03:09:32.125Z 10.0.1.4  60771.0       10.0.1.6       5985.0                     1   
                                                                  60772.0       10.0.1.6       5985.0                     1   
                                                                  60774.0       10.0.1.6       5985.0                     1   
                                                                  60775.0       10.0.1.6       5985.0                     1   
                               2020-05-02T03:09:33.128Z 10.0.1.4  60771.0       10.0.1.6       5985.0                     1   
                                                                  60772.0       10.0.1.6       5985.0                     1   
                                                                  60774.0       10.0.1.6       5985.0                     1   
                                                                  60775.0       10.0.1.6       5985.0                     1   
simplerockbuild.simplerock.lan 2020-05-02T02:56:00.768Z 10.0.1.4  60597.0       192.168.0.5    1234.0                     1   
                               2020-05-02T02:56:02.783Z 10.0.1.4  60597.0       192.168.0.5    1234.0                     1   
                               2020-05-02T02:58:4

In [51]:
#df_zeek2.to_json(path_or_buf='./combined_zeek_labelled.log', orient='records')

In [52]:
#df_zeek3 = pd.read_json("combined_zeek_labelled.log")
#print(type(df_zeek3))
#df_zeek3.head()

In [53]:
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql.types import *
import pyspark.sql.functions as func
from pyspark.sql.functions import length, concat ,concat_ws

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.caseSensitive", "true")
spark.sparkContext.setCheckpointDir("./tmp/checkpoints")
spark.sparkContext.version
spark.conf.set("spark.driver.memory", "3g")
spark.conf.set("spark.executor.memory", "3g")

In [54]:
df_labelled = spark.createDataFrame(df_zeek2.astype(str))
#df=spark.createDataFrame(pdDF.astype(str)) 

In [55]:
df_labelled.coalesce(1).write.format('json').save('./df_zeek_labelled3')

In [57]:
df_zeek3 = pd.read_json('./df_zeek_labelled3.json', lines=True)
print(type(df_zeek3))
df_zeek3.head()

<class 'pandas.core.frame.DataFrame'>


,@stream,@system,@proc,ts,uid,id_orig_h,id_orig_p,id_resp_h,id_resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,orig_l2_addr,resp_l2_addr,rtt,named_pipe,endpoint,operation,trans_id,query,qclass,qclass_name,qtype,qtype_name,rcode,rcode_name,AA,TC,RD,RA,Z,answers,TTLs,rejected,analyzer,failure_reason,fuid,tx_hosts,rx_hosts,conn_uids,source,depth,analyzers,mime_type,filename,is_orig,seen_bytes,total_bytes,missing_bytes,overflow_bytes,timedout,md5,sha1,request_type,client,success,error_msg,till,forwardable,renewable,cipher,note,msg,sub,src,dst,p,actions,suppress_for,action,path,name,size,times_modified,times_accessed,times_created,times_changed,share_type,version,curve,server_name,resumed,established,cert_chain_fuids,client_cert_chain_fuids,subject,issuer,validation_status,resp_certificate_sha1,ja3,ja3s,notice,peer,addl,id,certificate_version,certificate_serial,certificate_subject,certificate_issuer,certificate_not_valid_before,certificate_not_valid_after,certificate_key_alg,certificate_sig_alg,certificate_key_type,certificate_key_length,certificate_exponent,basic_constraints_ca,extracted,extracted_cutoff,trans_depth,method,host,uri,user_agent,request_body_len,response_body_len,status_code,status_msg,tags,resp_fuids,resp_mime_types,client_header_names,orig_fuids,orig_mime_types,machine,compile_ts,os,subsystem,is_exe,is_64bit,uses_aslr,uses_dep,uses_code_integrity,uses_seh,has_import_table,has_export_table,has_cert_table,has_debug_data,section_names,Source_IP,Source_Port_x,Destination_IP,Destination_Port,ts_pd,EventID,TP_Binary,TP_Label,@timestamp,Source_Port_y
0,conn,bobs.bigwheel.local,zeek,1.588205e+09,Cvf4XX17hSAgXDdGEd,10.0.1.6,54243.0,10.0.0.4,53.0,udp,dns,0.001528,39.0,55.0,SF,0.0,Dd,1.0,67.0,1.0,83.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,nan,nan,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,NaN,NaN,NaN,NaN,nan,nan,NaN,nan,nan,nan,nan,nan,nan,nan,NaN,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,NaN,nan,NaN,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,NaN,nan,NaN,nan,nan,nan,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,10.0.1.6,54243.0,10.0.0.4,53.0,2020-04-30 00:06:39.824369907+00:00,NaN,False,0,nan,NaN
1,conn,bobs.bigwheel.local,zeek,1.588205e+09,CJ21Le4zsTUcyKKi98,10.0.1.6,56880.0,10.0.0.4,445.0,tcp,"gssapi,smb,krb",10.761077,4020.0,1681.0,RSTO,0.0,ShADdaR,12.0,1307.0,11.0,2133.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,nan,nan,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,NaN,NaN,NaN,NaN,nan,nan,NaN,nan,nan,nan,nan,nan,nan,nan,NaN,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,NaN,nan,NaN,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,NaN,nan,NaN,nan,nan,nan,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,10.0.1.6,56880.0,10.0.0.4,445.0,2020-04-30 00:06:39.827696085+00:00,NaN,False,0,nan,NaN
2,conn,bobs.bigwheel.local,zeek,1.588205e+09,CnOP7t1eGGHf6LFfuk,10.0.1.6,65108.0,10.0.0.4,53.0,udp,dns,0.001599,36.0,104.0,SF,0.0,Dd,1.0,64.0,1.0,132.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,nan,nan,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,NaN,NaN,NaN,NaN,nan,nan,NaN,nan,nan,nan,nan,nan,nan,nan,NaN,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,NaN,nan,NaN,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,NaN,nan,NaN,nan,nan,nan,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,10.0.1.6,65108.0,10.0.0.4,53.0,2020-04-30 00:07:59.953876019+00:00,

In [58]:
df_zeek3['@timestamp'].value_counts()

nan                         2114
2020-05-02T03:09:31.065Z      16
2020-05-02T03:09:07.649Z      12
2020-05-02T03:09:05.633Z      12
2020-05-02T03:09:07.650Z      12
2020-05-02T03:09:05.635Z      12
2020-05-02T02:58:48.298Z      10
2020-05-02T03:09:33.128Z       8
2020-05-02T03:09:32.125Z       8
2020-05-02T03:09:29.935Z       8
2020-05-02T03:09:24.813Z       8
2020-05-02T03:09:28.850Z       8
2020-05-02T03:09:27.836Z       8
2020-05-02T02:58:47.196Z       5
2020-05-02T02:58:47.195Z       5
2020-05-02T02:56:02.783Z       1
2020-05-02T02:56:00.768Z       1
Name: @timestamp, dtype: int64

In [59]:
df_zeek3.shape

(2248, 158)